In [59]:
import pandas as pd
import pymongo

In [60]:
# 連到本地端 mongo
client = pymongo.MongoClient(host='mongodb://localhost:27017/', port=27017)

In [61]:
# 沒有跳出 timeouterror 代表有連線成功
client.server_info()

{'version': '4.2.6',
 'gitVersion': '20364840b8f1af16917e4c23c1b5f5efd8b352f8',
 'modules': [],
 'allocator': 'system',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 2, 6, 0],
 'openssl': {'running': 'Apple Secure Transport'},
 'buildEnvironment': {'distmod': '',
  'distarch': 'x86_64',
  'cc': '/Applications/Xcode10.2.0.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/clang: Apple LLVM version 10.0.1 (clang-1001.0.46.3)',
  'ccflags': '-isysroot /Applications/Xcode10.2.0.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk -mmacosx-version-min=10.12 -target darwin16.0.0 -arch x86_64 -fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-unused-private-field -Wno-deprecated-declarations -Wno-tautological-constant-out-of-range-compare -Wno-tautological-constant-compare -Wno-tautological-u

In [62]:
# 擷取 database
database = client['test']

In [63]:
# 擷取 collections(table)
collection = database['col_test']

In [64]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test'), 'col_test')

In [65]:
# truncate collection(table)
collection.remove()

# 插入資料
doc1 = {"name":"Lucy", "sex":"female","job":"nurse", "salary":50}
doc2 = {"name":"Jerry", "sex":"male","job":"doctor", "salary":100}
doc3 = {"name":"Wendell", "sex":"male","job":"Engineer", "salary":200}
doc4 = {"name":"Damon", "sex":"male","job":"Engineer", "salary":150}
doc5 = {"name":"Anne", "sex":"female","job":"art", "salary":120}
collection.insert([doc1, doc2, doc3, doc4, doc5])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # Remove the CWD from sys.path while we load stuff.


[ObjectId('5ec231d7f203cf12cc07dc3f'),
 ObjectId('5ec231d7f203cf12cc07dc40'),
 ObjectId('5ec231d7f203cf12cc07dc41'),
 ObjectId('5ec231d7f203cf12cc07dc42'),
 ObjectId('5ec231d7f203cf12cc07dc43')]

In [66]:
# 這個 find() 只會返還一個 cursor, 必須用 for loop 把資料都取出來。g
data = collection.find() # 會返回一個 cursor

In [67]:
for d in data:
    print(d)

{'_id': ObjectId('5ec231d7f203cf12cc07dc3f'), 'name': 'Lucy', 'sex': 'female', 'job': 'nurse', 'salary': 50}
{'_id': ObjectId('5ec231d7f203cf12cc07dc40'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}
{'_id': ObjectId('5ec231d7f203cf12cc07dc41'), 'name': 'Wendell', 'sex': 'male', 'job': 'Engineer', 'salary': 200}
{'_id': ObjectId('5ec231d7f203cf12cc07dc42'), 'name': 'Damon', 'sex': 'male', 'job': 'Engineer', 'salary': 150}
{'_id': ObjectId('5ec231d7f203cf12cc07dc43'), 'name': 'Anne', 'sex': 'female', 'job': 'art', 'salary': 120}


In [68]:
# 查找特定值
data = collection.find({"name":"Jerry"})
for d in data:
    print(d)

{'_id': ObjectId('5ec231d7f203cf12cc07dc40'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}


In [72]:
# select * from collection where sex=female and salary>100
data = collection.find({'$and':[
            {'sex': 'female'},
            {'salary': {'$gt':100}} # gt for greater than; gte for greater than & equal to
            ]
        })
for d in data:
    print(d)

{'_id': ObjectId('5ec231d7f203cf12cc07dc43'), 'name': 'Anne', 'sex': 'female', 'job': 'art', 'salary': 120}


In [69]:
# 限制查詢個數
data = collection.find({"sex":"male"}).limit(2)
for d in data:
    print(d)

{'_id': ObjectId('5ec231d7f203cf12cc07dc40'), 'name': 'Jerry', 'sex': 'male', 'job': 'doctor', 'salary': 100}
{'_id': ObjectId('5ec231d7f203cf12cc07dc41'), 'name': 'Wendell', 'sex': 'male', 'job': 'Engineer', 'salary': 200}


In [70]:
# 直接把 cursor 結果變成一個 list or dataframe
data = collection.find()
df = list(data)
pd.DataFrame(df)

,_id,name,sex,job,salary
0,5ec231d7f203cf12cc07dc3f,Lucy,female,nurse,50
1,5ec231d7f203cf12cc07dc40,Jerry,male,doctor,100
2,5ec231d7f203cf12cc07dc41,Wendell,male,Engineer,200
3,5ec231d7f203cf12cc07dc42,Damon,male,Engineer,150
4,5ec231d7f203cf12cc07dc43,Anne,female,art,120


***

In [65]:
MONGO_HOST = "mongos-auth.test.intl.imvideo.tv:27017"
# MONGO_DB = "DATABASE_NAME"
MONGO_USER = "im_data_user"
MONGO_PASS = "a945dcc8510a19883a367be9b3f41c2fa81e5f9b"

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()

client = pymongo.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]
# print(db.collection_names())

# server.stop()

2020-05-18 13:44:23,213| ERROR   | Could not resolve IP address for mongos-auth.test.intl.imvideo.tv:27017, aborting!


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [61]:
client = pymongo.MongoClient(host='mongodb://mongos-auth.test.intl.imvideo.tv:27017/', port=27017)

In [62]:
# check if connected
client.server_info()

ServerSelectionTimeoutError: mongos-auth.test.intl.imvideo.tv:27017: [Errno 8] nodename nor servname provided, or not known

In [34]:
database_im = client['im_internal']

In [35]:
collection = database_im['recording_like']

In [36]:
collection.stats

Collection(Database(MongoClient(host=['mongos-auth.test.intl.imvideo.tv:27017'], document_class=dict, tz_aware=False, connect=True), 'im_internal'), 'recording_like.stats')

In [37]:
collection.find_one()

ServerSelectionTimeoutError: mongos-auth.test.intl.imvideo.tv:27017: [Errno 8] nodename nor servname provided, or not known